# **과제1 : 챗봇만들기**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기1
    * Vector DB 준비하기
    * Retriever, LLM를 연결하기

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime
####################3주석처리####################
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

sk-proj-ZVcS6vWbnWT22uFwBxj0T3BlbkFJLkdrQtXo4thJtYbotXw1


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [3]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩

In [4]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
data.head()

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...


* 벡터 데이터베이스
    * Embedding 모델 : text-embedding-ada-002
    * DB 경로 : ./db



In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./db",  # 경로 지정(현 위치에서 db 폴더 생성)
                    embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* 데이터 입력
    * DF to Vector DB

In [6]:
# 데이터프레임의 텍스트 열(시리즈)을 리스트로 변환
text_list = data['QA'].tolist()

# 리스트 내용을 각각 document로 변환
documents = [Document(page_content=text) for text in text_list]

print(documents)

[Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.'), Document(page_content='35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)'), Document(page_content='미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함이 있는 근로인 경우,\n2) 고용보험에 미가입한 경우,\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.'), Document(page_content='직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.'), Document(page_content='아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\n다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로,\n고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기

In [7]:
# Insert
database.add_documents(documents)

['ba58fe27-39f5-4dac-b831-81f499c337a8',
 '23569a38-fe76-4919-a18d-1d41ac3650c7',
 '6a141f90-45a6-43b5-a746-e813fb89abf8',
 '1a9ea5b5-0001-44f0-a0c0-5d3b2019e47e',
 '9e4a1ead-8a52-48dd-b05f-a44e85ca0aa0',
 '0f5f48a2-b4b9-4c5f-bd83-8422c3fd7f32',
 'aded33f9-160a-47ae-a37a-3bbdf008ef43',
 '8288eb10-54e1-4291-aa8e-71c21333e92d',
 '1bb298ca-be14-435f-847a-e264de23b58f',
 'ebde8889-bc25-4f5f-9f6e-f763485f0359']

* 입력된 데이터 조회

In [8]:
database.get()

{'ids': ['0ccb8e7f-cd79-48dd-9b21-2ac62a60a36c',
  '0f5f48a2-b4b9-4c5f-bd83-8422c3fd7f32',
  '16d34a82-4c63-4ab4-b91a-7dd3ac0d9ded',
  '19b71bba-7ca1-48f6-8a21-238edc381a1b',
  '1a9ea5b5-0001-44f0-a0c0-5d3b2019e47e',
  '1bb298ca-be14-435f-847a-e264de23b58f',
  '2198b5d5-409a-429a-b73b-b7d310ebda1a',
  '2311fc92-8bee-4262-9337-47855cce18f9',
  '23569a38-fe76-4919-a18d-1d41ac3650c7',
  '27c65efb-78d5-4805-83c0-7d60fd31d06a',
  '396df38f-b4be-4a3b-8c17-a7f4f2613514',
  '3ac7c97c-17f8-4fc5-ac32-cf8c114a7e89',
  '3edd3555-ffbb-4b2a-9537-f12416eb9ad6',
  '44d86b0e-9d08-4d0d-b98c-3a7fd6d3d89b',
  '47e5a585-b256-4a2e-8b00-9d97285c0f44',
  '68ced5ca-7992-4fe2-b4f9-bf0f05a95ed7',
  '6a141f90-45a6-43b5-a746-e813fb89abf8',
  '6df6bb0a-7ce2-4cc1-b9a0-42e3f88dc509',
  '702f2f82-11e9-4784-9868-b037610d7cac',
  '7ebc7c0d-a820-4aa9-8a5f-8df1df194d16',
  '8288eb10-54e1-4291-aa8e-71c21333e92d',
  '87248fcc-1bcf-4bbd-96a0-91f8f23666be',
  '8c8a458e-a001-4590-8b94-1653cc46dc8b',
  '8cdfaa81-7f3f-4d60-98ca-

## **3.RAG+LLM모델**

* 모델 : RetrievalQA
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정

* database를 retriever로 선언하기

In [9]:
k =3
chat = ChatOpenAI(model="gpt-3.5-turbo")
retriever = database.as_retriever(search_kwargs={"k": k})

* 모델 선언

In [15]:
query = input("쿼리를 입력하세요: ")
#query = "최종 학력 또는 전공과 관계없이 지원할 수 있나요?"
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)

* 모델 확인

In [17]:
print(result)

print('-'*50)
result2 = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result2)
print('-'*50)
for doc in result2:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

{'query': '최종 학력 또는 전공과 관계없이 지원할 수 있나요?', 'result': 'KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육이지만, 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.', 'source_documents': [Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.'), Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.'), Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.')]}
--------------------------------------------------
[(Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는 교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.'), 0.18964090038306922), (Document